## Новый подход

In [1]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional, Union, Generator
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, save_npz, load_npz
import json
import pickle
from pathlib import Path
import psutil
import os
import gc
import time
from dataclasses import dataclass
import warnings
import re
warnings.filterwarnings('ignore')


def clean_text_for_embedding(text):
    """
    Быстрая чистка текста для эмбеддингов
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Удаляем ссылки и email
    text = re.sub(r'http\S+|www\S+|https\S+|@\S+', '', text, flags=re.MULTILINE)
    
    # 2. Удаляем специальные символы и цифры в скобках (например, (1), (2))
    text = re.sub(r'\(\d+\)', '', text)
    
    # 3. Удаляем подписи подкастов и списки
    text = re.sub(r'Subscribe to.*casts\.', '', text, flags=re.DOTALL)
    text = re.sub(r'Listen to.*episode.*:', '', text, flags=re.DOTALL)
    text = re.sub(r'You also can follow.*@\w+', '', text)
    
    # 4. Удаляем множественные переносы строк
    text = re.sub(r'\n\s*\n', '\n', text)
    text = re.sub(r'\n+', ' ', text)
    
    # 5. Удаляем лишние пробелы
    text = ' '.join(text.split())
    
    return text.strip()


@dataclass
class EmbeddingConfig:
    """Конфигурация для генерации эмбеддингов"""
    batch_size: int = 32
    use_mmap: bool = True  # Использовать memory-mapped файлы для больших данных
    temp_dir: str = "./temp_embeddings"
    max_memory_gb: float = 4.0  # Максимальное использование памяти в GB
    dtype: str = "float32"

class BaseEmbeddingGenerator(ABC):
    """Базовый абстрактный класс для генераторов эмбеддингов"""
    
    def __init__(self, config: EmbeddingConfig = None):
        self.config = config or EmbeddingConfig()
        Path(self.config.temp_dir).mkdir(parents=True, exist_ok=True)
        
    @abstractmethod
    def fit(self, texts: List[str]) -> 'BaseEmbeddingGenerator':
        """Обучение модели на текстах"""
        pass
    
    @abstractmethod
    def transform(self, texts: List[str]) -> Union[np.ndarray, csr_matrix]:
        """Преобразование текстов в эмбеддинги"""
        pass
    
    @abstractmethod
    def fit_transform(self, texts: List[str]) -> Union[np.ndarray, csr_matrix]:
        """Обучение и преобразование"""
        pass
    
    def save(self, path: str) -> None:
        """Сохранение модели"""
        save_path = Path(path)
        save_path.mkdir(parents=True, exist_ok=True)
        
        # Сохраняем конфиг
        config_path = save_path / "config.json"
        with open(config_path, 'w') as f:
            json.dump(self.config.__dict__, f)
    
    @classmethod
    def load(cls, path: str) -> 'BaseEmbeddingGenerator':
        """Загрузка модели"""
        pass
    
    def _check_memory_usage(self) -> bool:
        """Проверка использования памяти"""
        process = psutil.Process(os.getpid())
        memory_gb = process.memory_info().rss / 1024 / 1024 / 1024
        
        if memory_gb > self.config.max_memory_gb:
            print(f"⚠️ Предупреждение: Использовано {memory_gb:.2f} GB памяти "
                  f"(лимит: {self.config.max_memory_gb} GB)")
            return False
        return True
    
    def _batch_generator(self, data: List, batch_size: int) -> Generator:
        """Генератор батчей"""
        for i in range(0, len(data), batch_size):
            yield data[i:i + batch_size]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

class BM25Vectorizer:
    """Оптимизированный BM25 векторайзер"""
    def __init__(self, k1: float = 1.5, b: float = 0.75, 
                 max_features: int = 50000, 
                 language: str = "russian",
                 use_stopwords: bool = True,
                 use_stemming: bool = False):
        self.k1 = k1
        self.b = b
        self.max_features = max_features
        self.language = language
        self.use_stopwords = use_stopwords
        self.use_stemming = use_stemming
        
        # Инициализация компонентов
        self.stemmer = SnowballStemmer(language) if use_stemming else None
        self.stop_words = set(stopwords.words(language)) if use_stopwords else None
        self.vectorizer = None
        self.avgdl = 0.0
        self.idf = None
        self.vocabulary_ = None
        
    def _preprocess_text(self, text: str) -> str:
        """Предобработка текста"""
        text = clean_text_for_embedding(text)
        if self.use_stemming and self.stemmer:
            tokens = word_tokenize(text.lower())
            tokens = [self.stemmer.stem(t) for t in tokens 
                     if t.isalpha() and len(t) > 2]
            if self.use_stopwords and self.stop_words:
                tokens = [t for t in tokens if t not in self.stop_words]
            return ' '.join(tokens)
        return text.lower()
    
    def fit(self, texts: List[str]):
        """Обучение BM25"""
        print("Обучение BM25 векторайзера...")
        
        # Предобработка
        processed_texts = [self._preprocess_text(t) for t in texts]
        
        # TF векторайзер
        self.vectorizer = CountVectorizer(
            max_features=self.max_features,
            stop_words=list(self.stop_words) if self.use_stopwords else None,
            lowercase=True
        )
        
        # Вычисление TF
        tf_matrix = self.vectorizer.fit_transform(processed_texts)
        self.vocabulary_ = self.vectorizer.vocabulary_
        
        # Вычисление среднего длины документа
        doc_lengths = tf_matrix.sum(axis=1).A1
        self.avgdl = doc_lengths.mean()
        
        # Вычисление IDF
        n_docs = tf_matrix.shape[0]
        df = (tf_matrix > 0).sum(axis=0).A1
        self.idf = np.log((n_docs - df + 0.5) / (df + 0.5) + 1.0)
        
        return self
    
    def transform(self, texts: List[str]) -> csr_matrix:
        """Преобразование текстов в BM25 векторы"""
        if self.vectorizer is None:
            raise ValueError("Сначала вызовите fit()")
        
        processed_texts = [self._preprocess_text(t) for t in texts]
        tf_matrix = self.vectorizer.transform(processed_texts)
        
        # Вычисление BM25
        doc_lengths = tf_matrix.sum(axis=1).A1
        tf = tf_matrix.tocsc()
        
        # Вычисление BM25 для каждого элемента
        k1 = self.k1
        b = self.b
        avgdl = self.avgdl
        
        # Вычисление score
        rows, cols = tf.nonzero()
        data = tf.data
        
        for i in range(len(data)):
            doc_len = doc_lengths[rows[i]]
            tf_val = data[i]
            idf_val = self.idf[cols[i]]
            
            numerator = tf_val * (k1 + 1)
            denominator = tf_val + k1 * (1 - b + b * doc_len / avgdl)
            
            data[i] = idf_val * numerator / denominator
        
        return tf.tocsr()

class SparseEmbeddingGenerator(BaseEmbeddingGenerator):
    """Генератор sparse эмбеддингов (BM25)"""
    
    def __init__(self, 
                 language: str = "russian",
                 use_stopwords: bool = True,
                 use_stemming: bool = False,
                 max_features: int = 50000,
                 bm25_k1: float = 1.5,
                 bm25_b: float = 0.75,
                 config: EmbeddingConfig = None):
        super().__init__(config)
        
        self.language = language
        self.use_stopwords = use_stopwords
        self.use_stemming = use_stemming
        self.max_features = max_features
        self.bm25_k1 = bm25_k1
        self.bm25_b = bm25_b
        
        self.bm25 = None
        self.vocabulary_size = 0
        
    def fit(self, texts: List[str]) -> 'SparseEmbeddingGenerator':
        """Обучение BM25 на текстах"""
        print(f"Обучение sparse модели (BM25) на {len(texts)} документах...")
        
        self.bm25 = BM25Vectorizer(
            k1=self.bm25_k1,
            b=self.bm25_b,
            max_features=self.max_features,
            language=self.language,
            use_stopwords=self.use_stopwords,
            use_stemming=self.use_stemming
        )
        
        self.bm25.fit(texts)
        self.vocabulary_size = len(self.bm25.vocabulary_)
        
        print(f"✓ Обучение завершено. Словарь: {self.vocabulary_size} токенов")
        return self
    
    def transform(self, texts: List[str], 
                  save_to_file: Optional[str] = None) -> csr_matrix:
        """Преобразование текстов в sparse эмбеддинги"""
        if self.bm25 is None:
            raise ValueError("Сначала вызовите fit()")
        
        print(f"Преобразование {len(texts)} текстов в sparse эмбеддинги...")
        
        # Используем батчи для больших данных
        if len(texts) > 10000 and self.config.use_mmap:
            return self._transform_large(texts, save_to_file)
        
        # Для небольших данных
        embeddings = self.bm25.transform(texts)
        
        if save_to_file:
            save_npz(save_to_file, embeddings)
            print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        return embeddings
    
    def _transform_large(self, texts: List[str], 
                        save_to_file: Optional[str]) -> csr_matrix:
        """Преобразование больших объемов данных"""
        print("Используем батчевую обработку для больших данных...")
        
        batch_size = self.config.batch_size * 10  # Увеличиваем для sparse
        temp_files = []
        
        for i, batch in enumerate(self._batch_generator(texts, batch_size)):
            if i % 10 == 0:
                print(f"  Обработано {i * batch_size} документов")
                self._check_memory_usage()
            
            # Преобразуем батч
            batch_emb = self.bm25.transform(batch)
            
            # Сохраняем на диск
            temp_file = Path(self.config.temp_dir) / f"sparse_batch_{i}.npz"
            save_npz(temp_file, batch_emb)
            temp_files.append(temp_file)
            
            # Очищаем память
            del batch_emb
            gc.collect()
        
        # Объединяем результаты
        print("Объединение результатов...")
        embeddings = self._merge_sparse_files(temp_files)
        
        # Сохраняем итоговый файл
        if save_to_file:
            save_npz(save_to_file, embeddings)
        
        # Удаляем временные файлы
        for temp_file in temp_files:
            temp_file.unlink()
        
        return embeddings
    
    def _merge_sparse_files(self, file_paths: List[Path]) -> csr_matrix:
        """Объединение sparse матриц с диска"""
        from scipy.sparse import vstack
        
        matrices = []
        for i, file_path in enumerate(file_paths):
            if i % 10 == 0:
                print(f"  Загрузка файла {i}/{len(file_paths)}")
            matrices.append(load_npz(file_path))
        
        print("  Объединение матриц...")
        result = vstack(matrices)
        
        # Очищаем память
        del matrices
        gc.collect()
        
        return result
    
    def fit_transform(self, texts: List[str], 
                     save_to_file: Optional[str] = None) -> csr_matrix:
        """Обучение и преобразование"""
        self.fit(texts)
        return self.transform(texts, save_to_file)
    
    def save(self, path: str) -> None:
        """Сохранение модели"""
        super().save(path)
        save_path = Path(path)
        
        if self.bm25:
            model_path = save_path / "bm25_model.pkl"
            with open(model_path, 'wb') as f:
                pickle.dump({
                    'bm25': self.bm25,
                    'vocabulary_size': self.vocabulary_size,
                    'language': self.language,
                    'max_features': self.max_features
                }, f)
    
    @classmethod
    def load(cls, path: str) -> 'SparseEmbeddingGenerator':
        """Загрузка модели"""
        load_path = Path(path)
        
        # Загружаем конфиг
        config_path = load_path / "config.json"
        with open(config_path, 'r') as f:
            config_dict = json.load(f)
        
        config = EmbeddingConfig(**config_dict)
        
        # Создаем экземпляр
        instance = cls(config=config)
        
        # Загружаем модель
        model_path = load_path / "bm25_model.pkl"
        if model_path.exists():
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)
                instance.bm25 = model_data['bm25']
                instance.vocabulary_size = model_data['vocabulary_size']
                instance.language = model_data['language']
                instance.max_features = model_data['max_features']
        
        return instance

In [3]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

class DenseEmbeddingGenerator(BaseEmbeddingGenerator):
    """Генератор dense эмбеддингов"""
    
    def __init__(self,
                 model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                 device: Optional[str] = None,
                 normalize: bool = True,
                 config: EmbeddingConfig = None):
        super().__init__(config)
        
        self.model_name = model_name
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.normalize = normalize
        
        # Ленивая загрузка модели
        self._model = None
        self.embedding_dim = 0
    
    @property
    def model(self) -> SentenceTransformer:
        """Ленивая загрузка модели"""
        if self._model is None:
            print(f"Загрузка dense модели: {self.model_name}")
            
            # Освобождаем память перед загрузкой
            self._free_memory()
            
            self._model = SentenceTransformer(
                self.model_name,
                device=self.device
            )
            
            # Получаем размерность эмбеддингов
            test_embedding = self._model.encode(["test"], 
                                               convert_to_numpy=True)
            self.embedding_dim = test_embedding.shape[1]
            
            print(f"✓ Модель загружена. Размерность эмбеддингов: {self.embedding_dim}")
        
        return self._model
    
    def _free_memory(self):
        """Освобождение памяти"""
        gc.collect()
        if self.device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    def fit(self, texts: List[str]) -> 'DenseEmbeddingGenerator':
        """Для dense моделей fit не требуется (но нужен для интерфейса)"""
        print("Dense модели не требуют обучения на данных")
        return self
    
    def transform(self, texts: List[str], 
                  save_to_file: Optional[str] = None,
                  show_progress: bool = True) -> np.ndarray:
        """Преобразование текстов в dense эмбеддинги"""
        print(f"Генерация dense эмбеддингов для {len(texts)} текстов...")
        
        # Для очень больших данных используем memory-mapped файлы
        if len(texts) > 50000 and self.config.use_mmap:
            return self._transform_large_mmap(texts, save_to_file, show_progress)
        
        # Для средних данных используем батчевую обработку
        if len(texts) > 1000:
            return self._transform_batched(texts, save_to_file, show_progress)
        
        # Для небольших данных
        return self._transform_small(texts, save_to_file, show_progress)
    
    def _transform_small(self, texts: List[str], 
                        save_to_file: Optional[str],
                        show_progress: bool) -> np.ndarray:
        """Обработка небольших данных"""
        embeddings = self.model.encode(
            texts,
            batch_size=self.config.batch_size,
            show_progress_bar=show_progress,
            convert_to_numpy=True,
            normalize_embeddings=self.normalize
        )
        
        if save_to_file:
            np.save(save_to_file, embeddings)
            print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        return embeddings
    
    def _transform_batched(self, texts: List[str], 
                          save_to_file: Optional[str],
                          show_progress: bool) -> np.ndarray:
        """Батчевая обработка средних данных"""
        print("Используем батчевую обработку...")
        
        all_embeddings = []
        batch_size = self.config.batch_size
        
        # Прогресс-бар
        if show_progress:
            pbar = tqdm(total=len(texts), desc="Генерация эмбеддингов")
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            
            # Кодируем батч
            batch_emb = self.model.encode(
                batch_texts,
                batch_size=batch_size,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=self.normalize
            )
            
            all_embeddings.append(batch_emb)
            
            if show_progress:
                pbar.update(len(batch_texts))
            
            # Периодически проверяем память
            if i % (batch_size * 10) == 0:
                self._check_memory_usage()
                gc.collect()
        
        if show_progress:
            pbar.close()
        
        # Объединяем все батчи
        embeddings = np.vstack(all_embeddings)
        
        if save_to_file:
            np.save(save_to_file, embeddings)
            print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        return embeddings
    
    def _transform_large_mmap(self, texts: List[str], 
                             save_to_file: Optional[str],
                             show_progress: bool) -> np.ndarray:
        """Обработка очень больших данных с memory-mapped файлами"""
        print("Используем memory-mapped файлы для больших данных...")
        
        if not save_to_file:
            raise ValueError("Для больших данных требуется указать save_to_file")
        
        # Создаем memory-mapped файл
        total_samples = len(texts)
        mmap_file = np.memmap(
            save_to_file,
            dtype=self.config.dtype,
            mode='w+',
            shape=(total_samples, self.embedding_dim)
        )
        
        batch_size = self.config.batch_size
        
        # Прогресс-бар
        if show_progress:
            pbar = tqdm(total=total_samples, desc="Генерация эмбеддингов")
        
        for i in range(0, total_samples, batch_size):
            batch_texts = texts[i:i + batch_size]
            
            # Кодируем батч
            batch_emb = self.model.encode(
                batch_texts,
                batch_size=batch_size,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=self.normalize
            )
            
            # Записываем в memory-mapped файл
            mmap_file[i:i + len(batch_texts)] = batch_emb
            
            if show_progress:
                pbar.update(len(batch_texts))
            
            # Очищаем память
            del batch_emb
            if i % (batch_size * 20) == 0:
                self._check_memory_usage()
                gc.collect()
                if self.device == 'cuda':
                    torch.cuda.empty_cache()
        
        if show_progress:
            pbar.close()
        
        # Сохраняем изменения
        mmap_file.flush()
        
        print(f"✓ Эмбеддинги сохранены в {save_to_file}")
        
        # Возвращаем как обычный numpy array (только для чтения)
        return np.load(save_to_file, mmap_mode='r')
    
    def fit_transform(self, texts: List[str], 
                     save_to_file: Optional[str] = None,
                     show_progress: bool = True) -> np.ndarray:
        """Обучение и преобразование"""
        self.fit(texts)
        return self.transform(texts, save_to_file, show_progress)
    
    def save(self, path: str) -> None:
        """Сохранение конфигурации модели"""
        super().save(path)
        save_path = Path(path)
        
        # Сохраняем информацию о модели
        model_info = {
            'model_name': self.model_name,
            'device': self.device,
            'normalize': self.normalize,
            'embedding_dim': self.embedding_dim
        }
        
        model_info_path = save_path / "model_info.json"
        with open(model_info_path, 'w') as f:
            json.dump(model_info, f)
    
    @classmethod
    def load(cls, path: str) -> 'DenseEmbeddingGenerator':
        """Загрузка конфигурации модели"""
        load_path = Path(path)
        
        # Загружаем конфиг
        config_path = load_path / "config.json"
        with open(config_path, 'r') as f:
            config_dict = json.load(f)
        
        config = EmbeddingConfig(**config_dict)
        
        # Загружаем информацию о модели
        model_info_path = load_path / "model_info.json"
        with open(model_info_path, 'r') as f:
            model_info = json.load(f)
        
        # Создаем экземпляр
        instance = cls(
            model_name=model_info['model_name'],
            device=model_info['device'],
            normalize=model_info['normalize'],
            config=config
        )
        
        instance.embedding_dim = model_info['embedding_dim']
        
        return instance

2026-01-22 14:58:56.834437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769093937.036216      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769093937.097086      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769093937.566004      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769093937.566050      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769093937.566053      55 computation_placer.cc:177] computation placer alr

In [4]:
class EmbeddingFactory:
    """Фабрика для создания и управления эмбеддингами"""
    
    @staticmethod
    def create_sparse_generator(config: Dict[str, Any] = None) -> SparseEmbeddingGenerator:
        """Создание sparse генератора"""
        default_config = {
            'language': 'russian',
            'use_stopwords': True,
            'use_stemming': False,
            'max_features': 50000,
            'bm25_k1': 1.5,
            'bm25_b': 0.75,
            'config': EmbeddingConfig()
        }
        
        if config:
            default_config.update(config)
        
        return SparseEmbeddingGenerator(
            language=default_config['language'],
            use_stopwords=default_config['use_stopwords'],
            use_stemming=default_config['use_stemming'],
            max_features=default_config['max_features'],
            bm25_k1=default_config['bm25_k1'],
            bm25_b=default_config['bm25_b'],
            config=default_config['config']
        )
    
    @staticmethod
    def create_dense_generator(config: Dict[str, Any] = None) -> DenseEmbeddingGenerator:
        """Создание dense генератора"""
        default_config = {
            'model_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
            'device': None,
            'normalize': True,
            'config': EmbeddingConfig(batch_size=32)
        }
        
        if config:
            default_config.update(config)
        
        return DenseEmbeddingGenerator(
            model_name=default_config['model_name'],
            device=default_config['device'],
            normalize=default_config['normalize'],
            config=default_config['config']
        )
    
    @staticmethod
    def create_hybrid_pipeline(sparse_config: Dict[str, Any] = None,
                              dense_config: Dict[str, Any] = None) -> Dict:
        """Создание пайплайна для гибридных эмбеддингов"""
        return {
            'sparse': EmbeddingFactory.create_sparse_generator(sparse_config),
            'dense': EmbeddingFactory.create_dense_generator(dense_config)
        }


In [6]:
with open('/kaggle/input/crunch/techcrunch_ai_5488_articles_20260112_1535.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)

texts = [article['text'] for article in articles]
texts_clean = [clean_text_for_embedding(text) for text in texts]
print(f"Загружено {len(texts)} статей")

# 2. Конфигурация
sparse_config = {
    'language': 'russian',
    'max_features': 30000,
    'use_stopwords': True,
    'config': EmbeddingConfig(
        batch_size=100,
        use_mmap=True,
        max_memory_gb=8.0
    )
}
# 'intfloat/multilingual-e5-large-instruct'
model_name = "Qwen/Qwen3-Embedding-0.6B"
dense_config = {
    'model_name': model_name,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    #'device': 'cpu',
    'config': EmbeddingConfig(
        batch_size=2,
        use_mmap=True,
        max_memory_gb=4.0
    )
}

dense_gen = EmbeddingFactory.create_dense_generator(dense_config)
dense_file = "/kaggle/working/dense_embeddings.npy"
        
dense_embeddings = dense_gen.fit_transform(
    texts_clean,
    save_to_file=dense_file,
    show_progress=True
)

Загружено 5488 статей
Dense модели не требуют обучения на данных
Генерация dense эмбеддингов для 5488 текстов...
Используем батчевую обработку...


Генерация эмбеддингов:   0%|          | 0/5488 [00:00<?, ?it/s]

Загрузка dense модели: Qwen/Qwen3-Embedding-0.6B
✓ Модель загружена. Размерность эмбеддингов: 1024
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.33 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: Использовано 8.34 GB памяти (лимит: 4.0 GB)
⚠️ Предупреждение: 

In [21]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from typing import List, Dict, Set
from sklearn.preprocessing import normalize
from datetime import datetime


with open('/kaggle/input/crunch/techcrunch_ai_5488_articles_20260112_1535.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
print(f"Статей: {len(articles)}")

with open('/kaggle/input/crunch2/questions.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)
print(f"Вопросов: {len(questions)}")

article_id_to_idx = {}
article_titles = []
for idx, article in enumerate(articles):
    article_id = article.get('id', str(idx))
    article_id_to_idx[article_id] = idx
    article_titles.append(article.get('title', ''))

dense_file = "/kaggle/working/dense_embeddings.npy"
dense_embeddings = np.load(dense_file)
print(f"Эмбеддинги: {dense_embeddings.shape}")

# Создаем эмбеддинги для названий
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer(model_name, device=device)

title_embeddings = model.encode(
    article_titles,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)
print(f"Эмбеддинги названий: {title_embeddings.shape}")

# Нормализуем эмбеддинги
article_embeddings_norm = normalize(dense_embeddings)
title_embeddings_norm = normalize(title_embeddings)

def enhanced_dense_search(query: str, k: int = 10, 
                         text_weight: float = 0.7,
                         title_weight: float = 0.3) -> List[str]:
    """
    Улучшенный поиск с учетом текста и названий статей
    
    Args:
        query: текст запроса
        k: количество результатов
        text_weight: вес текста статьи (0-1)
        title_weight: вес названия статьи (0-1)
    """
    # Получаем эмбеддинг запроса
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Поиск по тексту статей
    text_similarities = query_emb @ article_embeddings_norm.T
    
    # Поиск по названиям статей
    title_similarities = query_emb @ title_embeddings_norm.T
    
    # Комбинируем результаты с весами
    combined_scores = (text_weight * text_similarities + 
                      title_weight * title_similarities)
    
    # Топ-K индексов
    top_indices = np.argsort(-combined_scores)[:k]
    
    # Преобразуем индексы в ID
    top_ids = []
    for idx in top_indices:
        # Ищем ID по индексу
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids
    
def enhanced_time_dense_search(
    query: str, 
    k: int = 10, 
    text_weight: float = 0.6,
    title_weight: float = 0.2,
    recency_weight: float = 0.2,
    time_decay_days: int = 365  # Статьи старше года получают минимальный буст
) -> List[str]:
    """
    Улучшенный поиск с учетом текста, названий и актуальности статей
    """
    # Получаем эмбеддинг запроса
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Поиск по тексту статей
    text_similarities = query_emb @ article_embeddings_norm.T
    
    # Поиск по названиям статей
    title_similarities = query_emb @ title_embeddings_norm.T
    
    # Рассчитываем временной скор
    current_time = datetime.now()
    time_scores = []
    
    for article in articles:
        try:
            pub_time = datetime.fromisoformat(article.get('published_time', '2020-01-01'))
            days_diff = (current_time - pub_time).days
            # Экспоненциальный затухающий буст (больше для свежих статей)
            time_score = np.exp(-days_diff / time_decay_days)
        except:
            time_score = 0.1  # Минимальный скор для статей без даты
        
        time_scores.append(time_score)
    
    time_scores = np.array(time_scores)
    time_scores = time_scores / time_scores.max()  # Нормализуем к [0, 1]
    
    # Комбинируем результаты
    combined_scores = (
        text_weight * text_similarities + 
        title_weight * title_similarities +
        recency_weight * time_scores
    )
    
    # Топ-K индексов
    top_indices = np.argsort(-combined_scores)[:k]
    
    # Преобразуем индексы в ID
    top_ids = []
    for idx in top_indices:
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids
    
def basic_dense_search(query: str, k: int = 10) -> List[str]:
    """Только по тексту статей"""
    query_emb = model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    similarities = query_emb @ article_embeddings_norm.T
    top_indices = np.argsort(-similarities)[:k]
    
    top_ids = []
    for idx in top_indices:
        for article_id, article_idx in article_id_to_idx.items():
            if article_idx == idx:
                top_ids.append(article_id)
                break
    
    return top_ids

# 4. Функция оценки
def evaluate_search(search_func, k_values=[1, 3, 5, 10]):
    """Оценка поиска с выводом метрик и деталей"""
    print(f"\n{'='*60}")
    print("ОЦЕНКА ПОИСКА ПО DENSE ЭМБЕДДИНГАМ")
    print(f"{'='*60}")
    
    results = []
    
    # Для каждого K считаем метрики
    for k in k_values:
        print(f"\nТОП-{k} РЕЗУЛЬТАТЫ:")
        print("-" * 80)
        
        total_precision = 0
        total_recall = 0
        total_hit = 0
        total_mrr = 0
        valid_queries = 0
        
        # Обрабатываем каждый вопрос
        for i, question_data in enumerate(questions):
            query = question_data.get("question", "")
            correct_ids = set(question_data.get("id", []))
            
            if not correct_ids:
                continue
            
            # Поиск
            found_ids = search_func(query, k=k)
            
            # Считаем метрики
            correct_found = [id for id in found_ids if id in correct_ids]
            
            # Precision@K
            precision = len(correct_found) / k if k > 0 else 0
            
            # Recall@K
            recall = len(correct_found) / len(correct_ids) if correct_ids else 0
            
            # Hit Rate@K
            hit = 1 if len(correct_found) > 0 else 0
            
            # MRR@K
            mrr = 0
            for rank, found_id in enumerate(found_ids, 1):
                if found_id in correct_ids:
                    mrr = 1.0 / rank
                    break
            
            # Добавляем к итогам
            total_precision += precision
            total_recall += recall
            total_hit += hit
            total_mrr += mrr
            valid_queries += 1
            
            # Выводим детали для первых 3 вопросов
            if i < 3:
                print(f"\nВопрос {i+1}: '{query[:50]}...'")
                print(f"Правильные ID: {list(correct_ids)}")
                print(f"Найденные ID: {found_ids}")
                print(f"Совпадения: {correct_found}")
                print(f"Несовпадения: {[id for id in found_ids if id not in correct_ids]}")
                print(f"Precision@{k}: {precision:.3f}, Recall@{k}: {recall:.3f}")
        
        # Средние метрики
        if valid_queries > 0:
            avg_precision = total_precision / valid_queries
            avg_recall = total_recall / valid_queries
            avg_hit = total_hit / valid_queries
            avg_mrr = total_mrr / valid_queries
            
            # F1-score
            f1 = 2 * avg_precision * avg_recall / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
            
            print(f"\n{'='*80}")
            print(f"ИТОГО ДЛЯ TOP-{k} (на {valid_queries} вопросах):")
            print(f"  Precision@{k}: {avg_precision:.3f}")
            print(f"  Recall@{k}:    {avg_recall:.3f}")
            print(f"  F1@{k}:        {f1:.3f}")
            print(f"  Hit Rate@{k}:  {avg_hit:.3f}")
            print(f"  MRR@{k}:       {avg_mrr:.3f}")
            print(f"{'='*80}")
            
            results.append({
                'k': k,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': f1,
                'hit_rate': avg_hit,
                'mrr': avg_mrr
            })
    
    return results


evaluation_results = evaluate_search(enhanced_time_dense_search, k_values=[1, 3, 5, 10])

print(f"\n{'='*80}")
print("СВОДНАЯ ТАБЛИЦА МЕТРИК")
print("="*80)
print(f"{'K':>4} {'Precision':>10} {'Recall':>10} {'F1':>10} {'Hit Rate':>10} {'MRR':>10}")
print("-" * 80)

for res in evaluation_results:
    print(f"{res['k']:>4} "
          f"{res['precision']:>10.3f} "
          f"{res['recall']:>10.3f} "
          f"{res['f1']:>10.3f} "
          f"{res['hit_rate']:>10.3f} "
          f"{res['mrr']:>10.3f}")

print("="*80)
safe_model_name = model_name.replace('/', '_') 
table_file = f"/kaggle/working/metrics_{safe_model_name}.txt"
with open(table_file, 'w', encoding='utf-8') as f:
    # Простая таблица
    f.write("Метрики поиска по dense эмбеддингам\n")
    f.write("="*60 + "\n")
    f.write("K  Precision  Recall     F1        HitRate   MRR\n")
    f.write("-" * 60 + "\n")
    
    for res in evaluation_results:
        f.write(f"{res['k']:2}  {res['precision']:8.3f}  {res['recall']:8.3f}  "
                f"{res['f1']:8.3f}  {res['hit_rate']:8.3f}  {res['mrr']:8.3f}\n")
    
    f.write("="*60 + "\n")

print(f"Таблица сохранена в {table_file}")

# 7. Детальный анализ для всех вопросов (сохранение в файл)
print("\nСОХРАНЕНИЕ ДЕТАЛЬНЫХ РЕЗУЛЬТАТОВ...")

detailed_results = []

for i, question_data in enumerate(questions):
    query = question_data.get("question", "")
    correct_ids = set(question_data.get("id", []))
    
    if not correct_ids:
        continue
    
    # Ищем для Top-5
    found_ids = enhanced_time_dense_search(query, k=5)
    
    # Определяем совпадения
    matches = [id for id in found_ids if id in correct_ids]
    non_matches = [id for id in found_ids if id not in correct_ids]
    
    # Precision для этого запроса
    precision_5 = len(matches) / 5 if found_ids else 0
    
    detailed_results.append({
        'query_id': i + 1,
        'query': query,
        'correct_ids': list(correct_ids),
        'found_ids': found_ids,
        'matches': matches,
        'non_matches': non_matches,
        'precision_5': precision_5,
        'num_matches': len(matches)
    })
    
    # Выводим детали для первых 5 вопросов
    if i < 5:
        print(f"\nВопрос {i+1}:")
        print(f"  Запрос: '{query[:60]}...'")
        print(f"  Правильные ID: {list(correct_ids)}")
        print(f"  Найденные ID: {found_ids}")
        print(f"  ✓ Совпали: {matches}")
        print(f"  ✗ Не совпали: {non_matches}")
        print(f"  Precision@5: {precision_5:.3f}")

safe_model_name = model_name.replace('/', '_') 
output_file = f"/kaggle/working/search_evaluation_details_{safe_model_name}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(detailed_results, f, indent=2, ensure_ascii=False)

print(f"\n✓ Детальные результаты сохранены в {output_file}")

# 8. Статистика совпадений
print(f"\n{'='*80}")
print("СТАТИСТИКА СОВПАДЕНИЙ (Top-5)")
print("="*80)

total_questions = len([q for q in questions if q.get('id')])
total_positions = total_questions * 5
total_matches = sum(res['num_matches'] for res in detailed_results)
total_non_matches = total_positions - total_matches

print(f"Всего вопросов с ответами: {total_questions}")
print(f"Всего проверок (вопросов × Top-5): {total_positions}")
print(f"Всего совпадений: {total_matches}")
print(f"Всего несовпадений: {total_non_matches}")
print(f"Процент совпадений: {total_matches/total_positions*100:.1f}%")

# Распределение по количеству совпадений
matches_dist = {}
for res in detailed_results:
    num_matches = res['num_matches']
    matches_dist[num_matches] = matches_dist.get(num_matches, 0) + 1

print(f"\nРаспределение совпадений на вопрос:")
for num in sorted(matches_dist.keys()):
    count = matches_dist[num]
    percentage = count / total_questions * 100
    print(f"  {num} совпадений: {count} вопросов ({percentage:.1f}%)")

print("="*80)

Статей: 5488
Вопросов: 60
Эмбеддинги: (5488, 1024)


Batches:   0%|          | 0/172 [00:00<?, ?it/s]

Эмбеддинги названий: (5488, 1024)

ОЦЕНКА ПОИСКА ПО DENSE ЭМБЕДДИНГАМ

ТОП-1 РЕЗУЛЬТАТЫ:
--------------------------------------------------------------------------------

Вопрос 1: 'What are the features that GPT-5 can provide compa...'
Правильные ID: ['69e1be8c-f789-5de3-8c1a-c89f92639178', '81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c', '13ed2979-8642-5bcd-809d-8897177f0c3d', '9a0df61d-4783-51f2-acc4-e66f52cd0813', '26a13d66-152b-5969-93e9-41e0b7b5c9aa']
Найденные ID: ['81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c']
Совпадения: ['81dd3690-0641-5fbd-b2a0-c2ae95bb9a0c']
Несовпадения: []
Precision@1: 1.000, Recall@1: 0.200

Вопрос 2: 'What new technologies in processor manufacturing w...'
Правильные ID: ['7ab20568-f68c-5cfe-a657-14c93f76771c', '8b678d66-56a3-55b5-9eb0-bcf11368c852', 'eb17f839-22c4-5c8e-ab21-32f423d32aa2', '821d70b8-720d-5430-8fe5-3f679ffdfdc7', 'ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7']
Найденные ID: ['ed6f3aa0-07bc-519e-b76a-aa79dc04b0a7']
Совпадения: ['ed6f3aa0-07bc-519e-b76a-aa79dc04b0

In [14]:
found_ids = enhanced_time_dense_search("What are AI agent systems, and how do they change the way automation is approached?", k=5)
print(found_ids)

['9c0a4b8e-704f-52f2-92ed-cc0c97da2c5d', '0ccf9b82-2a5d-522a-9eb2-c12eb8839587', 'fc60f3bc-4d99-595a-96bb-89b1afb489de', 'f837a58a-1a05-5673-885d-8eb9255426e4', 'df809136-d2b5-56a1-8300-0796991cc6e3']


## Проверка на размеченных данных

In [7]:
class SearchEvaluator:
    """
    Оценка качества поиска с правильными метриками
    """
    
    def __init__(self, questions_path, article_id_to_index):
        with open(questions_path, 'r') as f:
            self.questions = json.load(f)
        self.article_id_to_index = article_id_to_index
    
    def evaluate_method(self, search_function, method_name, k_values=[1, 3, 5, 10]):
        """
        Оценка метода поиска с разными значениями K
        
        Args:
            search_function: функция поиска (query, k) -> список результатов
            method_name: название метода
            k_values: значения K для расчета Precision@K, Recall@K и т.д.
        """
        print(f"\n{'='*60}")
        print(f"ОЦЕНКА МЕТОДА: {method_name}")
        print(f"{'='*60}")
        
        results = {
            "method": method_name,
            "total_queries": len(self.questions),
            "metrics": {}
        }
        
        # Инициализируем метрики для каждого K
        for k in k_values:
            results["metrics"][k] = {
                "precision": [],
                "recall": [],
                "f1": [],
                "hit_rate": [],
                "mrr": [],
                "ndcg": []  # Нормализованный дисконтированный кумулятивный выигрыш
            }
        
        # Обрабатываем каждый запрос
        for query_idx, query_data in enumerate(self.questions):
            query = query_data.get("question", "")
            relevant_ids = set(query_data.get("id", []))
            
            if not relevant_ids:
                continue  # Пропускаем вопросы без релевантных статей
            
            # Получаем результаты поиска для максимального K
            max_k = max(k_values)
            try:
                search_results = search_function(query, k=max_k)
            except Exception as e:
                print(f"  Ошибка для запроса {query_idx+1}: {e}")
                continue
            
            # Для каждого значения K считаем метрики
            for k in k_values:
                # Берем только топ-K результатов
                top_k_results = search_results[:k] if len(search_results) > k else search_results
                
                # Считаем метрики для этого K
                metrics = self._calculate_metrics_for_k(top_k_results, relevant_ids, k)
                
                # Добавляем к общим результатам
                for metric_name, value in metrics.items():
                    results["metrics"][k][metric_name].append(value)
            
            # Прогресс
            if (query_idx + 1) % 10 == 0:
                print(f"  Обработано {query_idx + 1}/{len(self.questions)} запросов")
        
        # Усредняем метрики по всем запросам
        print(f"\nРЕЗУЛЬТАТЫ ДЛЯ МЕТОДА '{method_name}':")
        print("-" * 80)
        
        summary_table = []
        for k in k_values:
            avg_metrics = {}
            for metric_name in results["metrics"][k]:
                values = results["metrics"][k][metric_name]
                if values:  # Если есть значения
                    avg_metrics[metric_name] = np.mean(values)
                else:
                    avg_metrics[metric_name] = 0.0
            
            summary_table.append({
                "K": k,
                "Precision": f"{avg_metrics['precision']:.3f}",
                "Recall": f"{avg_metrics['recall']:.3f}",
                "F1": f"{avg_metrics['f1']:.3f}",
                "Hit Rate": f"{avg_metrics['hit_rate']:.3f}",
                "MRR": f"{avg_metrics['mrr']:.3f}"
            })
            
            # Выводим для Top-5 более подробно
            if k == 5:
                print(f"Top-{k} Performance:")
                print(f"  Precision@{k}: {avg_metrics['precision']:.3f} - доля релевантных в топ-{k}")
                print(f"  Recall@{k}:    {avg_metrics['recall']:.3f} - доля найденных релевантных")
                print(f"  F1@{k}:        {avg_metrics['f1']:.3f} - баланс точности и полноты")
                print(f"  Hit Rate@{k}:  {avg_metrics['hit_rate']:.3f} - вероятность найти хоть что-то")
                print(f"  MRR@{k}:       {avg_metrics['mrr']:.3f} - качество ранжирования")
        
        # Выводим таблицу
        print(f"\nСводная таблица:")
        print("-" * 80)
        headers = ["K", "Precision", "Recall", "F1", "Hit Rate", "MRR"]
        row_format = "{:>4} {:>10} {:>10} {:>10} {:>10} {:>10}"
        print(row_format.format(*headers))
        print("-" * 80)
        for row in summary_table:
            print(row_format.format(
                row["K"], 
                row["Precision"], 
                row["Recall"], 
                row["F1"], 
                row["Hit Rate"], 
                row["MRR"]
            ))
        
        return results
    
    def _calculate_metrics_for_k(self, results, relevant_ids, k):
        """
        Расчет метрик для топ-K результатов
        
        Args:
            results: список результатов поиска (топ-K)
            relevant_ids: множество ID релевантных статей
            k: значение K (для нормализации)
        
        Returns:
            Словарь с метриками
        """
        # 1. Precision@K: доля релевантных в результатах
        relevant_found = 0
        for result in results:
            if result["article"]["id"] in relevant_ids:
                relevant_found += 1
        precision = relevant_found / len(results) if results else 0
        
        # 2. Recall@K: доля найденных релевантных от всех релевантных
        recall = relevant_found / len(relevant_ids) if relevant_ids else 0
        
        # 3. F1@K: гармоническое среднее
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # 4. Hit Rate@K: была ли найдена хоть одна релевантная статья
        hit_rate = 1 if relevant_found > 0 else 0
        
        # 5. MRR@K: средняя обратная позиция первого релевантного результата
        mrr = 0
        for rank, result in enumerate(results, 1):
            if result["article"]["id"] in relevant_ids:
                mrr = 1.0 / rank
                break
        
        # 6. NDCG@K (опционально, более сложная метрика)
        # Для простоты можно пропустить
        
        return {
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "hit_rate": hit_rate,
            "mrr": mrr
        }
    
    def compare_methods(self, methods_dict, k_values=[1, 3, 5, 10]):
        """
        Сравнение нескольких методов поиска
        
        Args:
            methods_dict: словарь {название_метода: функция_поиска}
            k_values: значения K для оценки
        """
        print("\n" + "="*80)
        print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ МЕТОДОВ ПОИСКА")
        print("="*80)
        
        all_results = {}
        
        for method_name, search_func in methods_dict.items():
            results = self.evaluate_method(search_func, method_name, k_values)
            all_results[method_name] = results
        
        # Ранжирование методов по F1@5
        print("\n" + "="*80)
        print("РАНЖИРОВАНИЕ МЕТОДОВ ПО F1@5:")
        print("="*80)
        
        rankings = []
        for method_name, results in all_results.items():
            f1_at_5 = np.mean(results["metrics"][5]["f1"]) if results["metrics"][5]["f1"] else 0
            precision_at_5 = np.mean(results["metrics"][5]["precision"]) if results["metrics"][5]["precision"] else 0
            recall_at_5 = np.mean(results["metrics"][5]["recall"]) if results["metrics"][5]["recall"] else 0
            
            rankings.append({
                "Method": method_name,
                "F1@5": f1_at_5,
                "Precision@5": precision_at_5,
                "Recall@5": recall_at_5
            })
        
        # Сортируем по F1@5
        rankings.sort(key=lambda x: x["F1@5"], reverse=True)
        
        # Выводим таблицу
        print("\nРейтинг методов (лучший → худший):")
        print("-" * 80)
        print(f"{'Метод':<20} {'F1@5':<8} {'Precision@5':<12} {'Recall@5':<10}")
        print("-" * 80)
        for rank, item in enumerate(rankings, 1):
            print(f"{rank}. {item['Method']:<18} {item['F1@5']:.3f}    {item['Precision@5']:.3f}        {item['Recall@5']:.3f}")
        
        return all_results


evaluator = SearchEvaluator("/kaggle/input/crunch/questions.json", article_id_to_index)

# Определяем методы поиска
# methods_to_test = {
#    "BM25": lambda q, k: hybrid_search.search_by_bm25(q, k=k),
#    "Dense": lambda q, k: hybrid_search.search_by_dense(q, k=k),
#    "Hybrid (α=0.3)": lambda q, k: hybrid_search.hybrid_search(q, k=k, alpha=0.3),
#    "Hybrid (α=0.5)": lambda q, k: hybrid_search.hybrid_search(q, k=k, alpha=0.5),
#    "Hybrid (α=0.7)": lambda q, k: hybrid_search.hybrid_search(q, k=k, alpha=0.7)
#}
methods_to_test = {"Dense": lambda q, k: hybrid_search.search_by_dense(q, k=k)}
# Полная оценка
full_results = evaluator.compare_methods(methods_to_test, k_values=[1, 3, 5, 10])

# Или быстрая оценка
print("\n" + "="*60)
print("БЫСТРАЯ ОЦЕНКА (только Top-5)")
print("="*60)

simple_evaluator = SimpleSearchEvaluator("/kaggle/input/crunch/questions.json", article_id_to_index)
quick_results = simple_evaluator.quick_compare(methods_to_test, k=5)

# Анализ примеров
print("\n" + "="*60)
print("АНАЛИЗ ПРИМЕРОВ РАБОТЫ ЛУЧШЕГО МЕТОДА")
print("="*60)

best_method_name = max(quick_results.items(), key=lambda x: x[1]['f1'])[0]
best_search_func = methods_to_test[best_method_name]

# Проанализируем несколько запросов
print(f"\nАнализ работы лучшего метода: {best_method_name}")
print("-" * 80)

sample_queries = evaluator.questions[:10]  # Первые 3 запроса

for i, query_data in enumerate(sample_queries):
    query = query_data.get("question", "")
    relevant_ids = set(query_data.get("id", []))
    
    print(f"\nЗапрос {i+1}: '{query[:60]}...'")
    print(f"Релевантные ID: {list(relevant_ids)[:3]}...")
    
    try:
        results = best_search_func(query, k=5)
        print(f"Найдено результатов: {len(results)}")
        
        print("Топ-5 результатов:")
        for j, result in enumerate(results[:5]):
            article_id = result["article"]["id"]
            is_relevant = "✓" if article_id in relevant_ids else "✗"
            score_key = "score" if "score" in result else "hybrid_score"
            score = result.get(score_key, 0)
            
            print(f"  {j+1}. [{is_relevant}] {result['article']['title'][:50]}... (score: {score:.3f})")
        
        # Считаем Precision@5 для этого запроса
        relevant_found = sum(1 for r in results if r["article"]["id"] in relevant_ids)
        precision = relevant_found / len(results) if results else 0
        print(f"  Precision@5 для этого запроса: {precision:.3f} ({relevant_found}/{len(results)})")
        
    except Exception as e:
        print(f"  Ошибка: {e}")

NameError: name 'article_id_to_index' is not defined

In [19]:
# 5. Упрощенная оценка
print("\n" + "="*60)
print("УПРОЩЕННАЯ ОЦЕНКА КАЧЕСТВА:")
print("="*60)

# Создаем простой evaluator
class SimpleEvaluator:
    def __init__(self, questions_path, article_id_to_index):
        with open(questions_path, 'r') as f:
            self.questions = json.load(f)
        self.article_id_to_index = article_id_to_index
    
    def evaluate(self, search_function, method_name, k=5):
        print(f"\n{method_name} (Top-{k}):")
        
        total_correct = 0
        total_possible = 0
        
        for query_data in self.questions:
            query = query_data["question"]
            relevant_ids = set(query_data.get("id", []))
            
            if not relevant_ids:
                continue
            
            try:
                results = search_function(query, k=k)
            except:
                continue
            
            # Считаем сколько релевантных нашли
            found_relevant = 0
            for result in results:
                if result["article"]["id"] in relevant_ids:
                    found_relevant += 1
            
            total_correct += found_relevant
            total_possible += min(len(relevant_ids), k)
        
        accuracy = total_correct / total_possible if total_possible > 0 else 0
        print(f"  Найдено релевантных: {total_correct}/{total_possible}")
        print(f"  Accuracy: {accuracy:.3f}")
        
        return accuracy

# Создаем mapping
article_id_to_index = {}
for idx, article in enumerate(results["articles"]):
    article_id_to_index[article["id"]] = idx

evaluator = SimpleEvaluator("/kaggle/input/crunch/questions.json", article_id_to_index)

# Инициализация поиска
search_data = {
    "articles": results["articles"],
    "bm25_matrix": results["bm25_matrix"],
    "dense_embeddings": results["dense_embeddings"],
    "vocabulary": results["vocabulary"],
    "bm25_vectorizer_info": results.get("bm25_vectorizer_info", {})
}

hybrid_search = HybridSearch(search_data)

# Оценка методов
methods = [
    ("BM25", lambda q, k: hybrid_search.search_by_bm25(q, k)),
    ("Dense", lambda q, k: hybrid_search.search_by_dense(q, k)),
    ("Hybrid α=0.3", lambda q, k: hybrid_search.hybrid_search(q, k, 0.3)),
    ("Hybrid α=0.5", lambda q, k: hybrid_search.hybrid_search(q, k, 0.5)),
    ("Hybrid α=0.7", lambda q, k: hybrid_search.hybrid_search(q, k, 0.7))
]

scores = {}
for method_name, search_func in methods:
    score = evaluator.evaluate(search_func, method_name, k=5)
    scores[method_name] = score

# Вывод лучшего метода
best_method = max(scores.items(), key=lambda x: x[1])
print(f"\n🎯 Лучший метод: {best_method[0]} с accuracy {best_method[1]:.3f}")


УПРОЩЕННАЯ ОЦЕНКА КАЧЕСТВА:

BM25 (Top-5):
  Найдено релевантных: 14/204
  Accuracy: 0.069

Dense (Top-5):
  Найдено релевантных: 24/204
  Accuracy: 0.118

Hybrid α=0.3 (Top-5):
  Найдено релевантных: 14/204
  Accuracy: 0.069

Hybrid α=0.5 (Top-5):
  Найдено релевантных: 18/204
  Accuracy: 0.088

Hybrid α=0.7 (Top-5):
  Найдено релевантных: 20/204
  Accuracy: 0.098

🎯 Лучший метод: Dense с accuracy 0.118


In [13]:
# 5. УПРОЩЕННАЯ ОЦЕНКА
print("\n" + "="*60)
print("УПРОЩЕННАЯ ОЦЕНКА КАЧЕСТВА:")
print("="*60)

# Сначала проверим данные
print("Проверка данных questions.json...")
with open("/kaggle/input/crunch/questions.json", 'r') as f:
    questions = json.load(f)

print(f"Загружено вопросов: {len(questions)}")
print("\nПример первых 2 вопросов:")
for i, q in enumerate(questions[:2]):
    print(f"  {i+1}. Вопрос: '{q.get('question', 'NO QUESTION')[:50]}...'")
    print(f"     ID релевантных статей: {q.get('id', 'NO ID')}")
    print()

# Создаем mapping
article_id_to_index = {}
for idx, article in enumerate(results["articles"]):
    article_id_to_index[article["id"]] = idx

print(f"Создан mapping для {len(article_id_to_index)} статей")
print(f"Пример ID статей: {list(article_id_to_index.keys())[:5]}")

# Проверим совпадение ID
print("\nПроверка совпадения ID вопросов и статей...")
for i, q in enumerate(questions[:3]):
    relevant_ids = q.get('id', [])
    if isinstance(relevant_ids, str):
        relevant_ids = [relevant_ids]
    
    for rel_id in relevant_ids:
        if rel_id in article_id_to_index:
            print(f"  ✓ Вопрос '{q.get('question', '')[:30]}...': ID {rel_id} найден в статьях")
        else:
            print(f"  ✗ Вопрос '{q.get('question', '')[:30]}...': ID {rel_id} НЕ найден в статьях")

# Создаем простой evaluator
class SimpleEvaluator:
    def __init__(self, questions_path, article_id_to_index):
        with open(questions_path, 'r') as f:
            self.questions = json.load(f)
        self.article_id_to_index = article_id_to_index
    
    def evaluate(self, search_function, method_name, top_k=5):
        print(f"\n{'='*40}")
        print(f"ТЕСТИРОВАНИЕ МЕТОДА: {method_name}")
        print(f"{'='*40}")
        
        total_correct = 0
        total_possible = 0
        
        # Протестируем сначала на одном запросе с детальным выводом
        test_query_data = self.questions[0] if self.questions else None
        if test_query_data:
            query = test_query_data.get("question", "")
            print(f"\nТестовый запрос: '{query}'")
            
            # Получаем результаты поиска
            try:
                search_results = search_function(query, top_k=top_k)
                print(f"  Найдено результатов: {len(search_results)}")
                
                if search_results:
                    print("  Первые 3 результата:")
                    for i, result in enumerate(search_results[:3]):
                        article_id = result["article"]["id"]
                        score = result.get("score", result.get("hybrid_score", 0))
                        print(f"    {i+1}. ID: {article_id}, Score: {score:.4f}, Title: {result['article']['title'][:50]}...")
                else:
                    print("  ⚠ Результаты поиска пустые!")
                    # Попробуем debug, почему поиск не работает
                    self._debug_search(search_function, query)
                    
            except Exception as e:
                print(f"  Ошибка при поиске: {e}")
                import traceback
                traceback.print_exc()
        
        # Теперь оценим все вопросы
        print(f"\nОценка на всех вопросах ({method_name}, Top-{top_k}):")
        
        for query_idx, query_data in enumerate(self.questions):
            query = query_data.get("question", "")
            # Внимание: у вас в JSON поле называется 'id', а не 'relevant_article_ids'
            relevant_ids = query_data.get("id", [])
            
            # Если это строка, делаем список
            if isinstance(relevant_ids, str):
                relevant_ids = [relevant_ids]
            
            if not relevant_ids:
                # print(f"  Вопрос {query_idx+1}: нет релевантных ID")
                continue
            
            try:
                results = search_function(query, top_k=top_k)
            except Exception as e:
                # print(f"  Вопрос {query_idx+1}: ошибка - {e}")
                continue
            
            # Считаем сколько релевантных нашли
            found_relevant = 0
            for result in results:
                if result["article"]["id"] in relevant_ids:
                    found_relevant += 1
            
            total_correct += found_relevant
            total_possible += min(len(relevant_ids), top_k)
            
            # if query_idx < 3:  # Показать первые 3 для debug
            #     print(f"  Вопрос {query_idx+1}: '{query[:30]}...'")
            #     print(f"    Релевантные ID: {relevant_ids}")
            #     print(f"    Найдено: {found_relevant}/{min(len(relevant_ids), top_k)}")
        
        accuracy = total_correct / total_possible if total_possible > 0 else 0
        print(f"\n  ИТОГО:")
        print(f"    Найдено релевантных: {total_correct}/{total_possible}")
        print(f"    Accuracy: {accuracy:.3f}")
        
        return accuracy
    
    def _debug_search(self, search_function, query):
        """Debug функция для понимания, почему поиск не работает"""
        print("\n  DEBUG поиска:")
        
        # Проверим, что это за функция
        print(f"    Тип search_function: {type(search_function)}")
        
        # Проверим, доступен ли hybrid_search
        try:
            # Если это лямбда, попробуем проверить ее работу
            import inspect
            print(f"    Сигнатура: {inspect.signature(search_function)}")
        except:
            pass

# Инициализация поиска
print("\n" + "="*60)
print("ИНИЦИАЛИЗАЦИЯ ПОИСКА")
print("="*60)

search_data = {
    "articles": results["articles"],
    "bm25_matrix": results["bm25_matrix"],
    "dense_embeddings": results["dense_embeddings"],
    "vocabulary": results["vocabulary"],
    "bm25_vectorizer_info": results.get("bm25_vectorizer_info", {})
}

print(f"Статей в search_data: {len(search_data['articles'])}")
print(f"BM25 матрица shape: {search_data['bm25_matrix'].shape}")
print(f"Dense эмбеддинги shape: {search_data['dense_embeddings'].shape}")

hybrid_search = HybridSearch(search_data)
print(f"✓ HybridSearch инициализирован")
print(f"  has_dense: {hybrid_search.has_dense}")

# Сначала протестируем поиск вручную на тестовых запросах
print("\n" + "="*60)
print("ТЕСТОВЫЕ ЗАПРОСЫ ВРУЧНУЮ")
print("="*60)

test_queries = [
    "Google AI medical queries",
    "Indonesia blocks chatbot",
    "deepfake regulations"
]

for query in test_queries:
    print(f"\nЗапрос: '{query}'")
    
    # BM25 поиск
    try:
        bm25_results = hybrid_search.search_by_bm25(query, top_k=3)
        print(f"  BM25: найдено {len(bm25_results)} результатов")
        if bm25_results:
            for i, r in enumerate(bm25_results[:2]):
                print(f"    {i+1}. {r['article']['title'][:50]}... (score: {r['score']:.4f})")
    except Exception as e:
        print(f"  BM25 ошибка: {e}")
    
    # Dense поиск
    if hybrid_search.has_dense:
        try:
            dense_results = hybrid_search.search_by_dense(query, top_k=3)
            print(f"  Dense: найдено {len(dense_results)} результатов")
            if dense_results:
                for i, r in enumerate(dense_results[:2]):
                    print(f"    {i+1}. {r['article']['title'][:50]}... (score: {r['score']:.4f})")
        except Exception as e:
            print(f"  Dense ошибка: {e}")

# Теперь оценка
evaluator = SimpleEvaluator("/kaggle/input/crunch/questions.json", article_id_to_index)

# Определяем методы поиска
def bm25_search(query, k):
    return hybrid_search.search_by_bm25(query, top_k=k)

def dense_search(query, k):
    return hybrid_search.search_by_dense(query, top_k=k)

def hybrid_search_alpha_03(query, k):
    return hybrid_search.hybrid_search(query, top_k=k, alpha=0.3)

def hybrid_search_alpha_05(query, k):
    return hybrid_search.hybrid_search(query, top_k=k, alpha=0.5)

def hybrid_search_alpha_07(query, k):
    return hybrid_search.hybrid_search(query, top_k=k, alpha=0.7)

methods = [
    ("BM25", bm25_search),
    ("Dense", dense_search),
    ("Hybrid α=0.3", hybrid_search_alpha_03),
    ("Hybrid α=0.5", hybrid_search_alpha_05),
    ("Hybrid α=0.7", hybrid_search_alpha_07)
]

scores = {}
for method_name, search_func in methods:
    score = evaluator.evaluate(search_func, method_name, top_k=5)
    scores[method_name] = score

# Вывод лучшего метода
if scores:
    best_method = max(scores.items(), key=lambda x: x[1])
    print(f"\n🎯 Лучший метод: {best_method[0]} с accuracy {best_method[1]:.3f}")
else:
    print("\n⚠ Не удалось оценить ни один метод!")


УПРОЩЕННАЯ ОЦЕНКА КАЧЕСТВА:
Проверка данных questions.json...
Загружено вопросов: 55

Пример первых 2 вопросов:
  1. Вопрос: 'How are tech giants addressing the issue of AI hal...'
     ID релевантных статей: ['0821507d-107b-526d-a4e6-03c7ffb90318', '507593c7-dd09-5647-8d9c-2cedb1ca3226', '6095f86a-b1b8-5b27-aafe-760252116ae1']

  2. Вопрос: 'What is the strategic importance of Physical AI an...'
     ID релевантных статей: ['81ab4cfd-cb69-56b5-a0a7-ca08218a7117', '57c9e8b1-1d91-5d22-af7f-9a64fa0548e6', '87179b33-ad32-5122-a0cc-41ed46ab011c']

Создан mapping для 5488 статей
Пример ID статей: ['0821507d-107b-526d-a4e6-03c7ffb90318', '0dcd2040-b0b6-5f1a-bbf5-44d36b181391', 'cb570756-1aa7-5602-8651-7ff6f00a6952', '1aecd534-13a9-597f-b5ec-cc79ae6ba8e9', '81ab4cfd-cb69-56b5-a0a7-ca08218a7117']

Проверка совпадения ID вопросов и статей...
  ✓ Вопрос 'How are tech giants addressing...': ID 0821507d-107b-526d-a4e6-03c7ffb90318 найден в статьях
  ✓ Вопрос 'How are tech giants addressing...': I

Traceback (most recent call last):
  File "/tmp/ipykernel_55/1194900496.py", line 62, in evaluate
    search_results = search_function(query, top_k=top_k)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: bm25_search() got an unexpected keyword argument 'top_k'
Traceback (most recent call last):
  File "/tmp/ipykernel_55/1194900496.py", line 62, in evaluate
    search_results = search_function(query, top_k=top_k)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: dense_search() got an unexpected keyword argument 'top_k'
Traceback (most recent call last):
  File "/tmp/ipykernel_55/1194900496.py", line 62, in evaluate
    search_results = search_function(query, top_k=top_k)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: hybrid_search_alpha_03() got an unexpected keyword argument 'top_k'
Traceback (most recent call last):
  File "/tmp/ipykernel_55/1194900496.py", line 62, in evaluate
    search_results = search_function(query, top_k=